In [ ]:

# Importieren der notwendigen Bibliotheken
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms, models
from PIL import Image
from pathlib import Path
import matplotlib.pyplot as plt


In [ ]:
class BrainTumorDataset(Dataset):
    def __init__(self, image_dir, transform=None):
        """
        Initialisiert den Dataset für Gehirntumor-Klassifikation.
        Args:
        - image_dir: Pfad zu den Bildern. Die Ordnerstruktur sollte "yes/" (Tumor) und "no/" (Healthy) enthalten.
        - transform: Transformationen für die Vorverarbeitung.
        """
        self.image_paths = []
        self.labels = []
        self.transform = transform

        # Pfade zu "yes" und "no"-Ordnern
        tumor_dir = Path('data/yes/*.jpg') 
        healthy_dir = Path('data/no/*.jpg') 

        # Bilder aus "yes" (Tumor) mit Label 1
        self.image_paths.extend(tumor_dir.rglob("*.jpg"))
        self.labels.extend([1] * len(list(tumor_dir.rglob("*.jpg"))))

        # Bilder aus "no" (Healthy) mit Label 0
        self.image_paths.extend(healthy_dir.rglob("*.jpg"))
        self.labels.extend([0] * len(list(healthy_dir.rglob("*.jpg"))))

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image_path = self.image_paths[idx]
        label = self.labels[idx]
        image = Image.open(image_path).convert("RGB")
        if self.transform:
            image = self.transform(image)
        return image, label


# Transformationen für die Datenvorbereitung
data_transforms = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])


In [ ]:

# Transfer Learning Modell mit ResNet-18
class TransferLearningModel(nn.Module):
    def __init__(self):
        super(TransferLearningModel, self).__init__()
        self.base_model = models.resnet18(pretrained=True)
        for param in self.base_model.parameters():
            param.requires_grad = False
        num_features = self.base_model.fc.in_features
        self.base_model.fc = nn.Sequential(
            nn.Linear(num_features, 128),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(128, 2)
        )

    def forward(self, x):
        return self.base_model(x)

# Initialisierung von Modell, Loss-Funktion und Optimizer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = TransferLearningModel().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.base_model.fc.parameters(), lr=0.001)


In [ ]:

# Trainings- und Validierungsprozess
def train_model(model, dataloaders, criterion, optimizer, num_epochs=10):
    """
    Trainiert das Modell und validiert es in jedem Epoch.
    Args:
    - model: Das zu trainierende Modell.
    - dataloaders: Dictionary mit 'train' und 'val' Dataladern.
    - criterion: Loss-Funktion.
    - optimizer: Optimizer für das Training.
    - num_epochs: Anzahl der Epochen.
    """
    for epoch in range(num_epochs):
        print(f"Epoch {epoch+1}/{num_epochs}")
        print("-" * 10)
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()
            else:
                model.eval()
            running_loss = 0.0
            running_corrects = 0
            for inputs, labels in dataloaders[phase]:
                inputs, labels = inputs.to(device), labels.to(device)
                optimizer.zero_grad()
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)
            print(f"{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}")
    return model


In [ ]:

# Laden des Datasets und Aufteilen in Trainings- und Validierungsdaten
image_dir = "path_to_images"  # Pfad zu den Bildern anpassen
dataset = BrainTumorDataset(image_dir, transform=data_transforms)
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])
dataloaders = {
    'train': DataLoader(train_dataset, batch_size=16, shuffle=True),
    'val': DataLoader(val_dataset, batch_size=16, shuffle=False)
}

# Training starten
model = train_model(model, dataloaders, criterion, optimizer, num_epochs=10)


In [ ]:

# Speichern des Modells
torch.save(model.state_dict(), "brain_tumor_model.pth")
print("Modell gespeichert.")
